# 7506 - Trabajo Práctico 2

---

## Introducción

### Librerías

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import matplotlib.pyplot as plt
from matplotlib import style

In [2]:
# Dataset Train preprocesado
train_ds = pd.read_csv('datasets/tp1-train.csv')
train_ds = train_ds.drop(['Unnamed: 0'], axis=1)
train_ds.head()

,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,2021-07-29,2021-09-06 00:00:00.000000000,-34.564800,-58.444631,Belgrano,Departamento,5.0,4.0,279.0,245.0,1000000.0,Espectacular torre en la mejor zona de Belgran...
1,2021-12-16,2022-02-09 00:00:00.000000000,-34.591359,-58.418809,Palermo,Departamento,2.0,1.0,42.0,42.0,89900.0,VENTA 2 AMBIENTES RECICLADO - PALERMO
2,2021-06-19,2021-09-01 15:14:13.531306906,-34.602632,-58.428399,Almagro,Departamento,2.0,1.0,46.0,42.0,119900.0,2 ambientes al frente con balcón. A estrenar.
3,2021-12-16,2022-02-09 00:00:00.000000000,-34.564290,-58.439272,Palermo,Departamento,3.0,2.0,69.0,58.0,175000.0,DEPARTAMENTO 3 Ambientes Palermo-Balcón C/Parr...
4,2021-12-16,2022-02-09 00:00:00.000000000,-34.630517,-58.377143,Barracas,Departamento,1.0,1.0,45.0,41.0,120000.0,Venta-Barracas-Mono divisible- MOCA


In [3]:
# Dataset Test preprocesado
test_ds = pd.read_csv('datasets/tp1-test.csv')
test_ds = test_ds.drop(['Unnamed: 0'], axis=1)
test_ds

,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,-34.589649,-58.434926,Palermo,Casa,4.0,3.0,140.0,140.0,530000.0,Casa - Palermo
1,-34.598508,-58.443084,Villa Crespo,Departamento,1.0,1.0,33.0,30.0,78000.0,Monoambiente Departamento a Estrenar - Villa C...
2,-34.582098,-58.448859,Palermo,Departamento,1.0,1.0,35.0,30.0,87100.0,Venta - Newbery Jorge 3445 3º I Jardin
3,-34.569662,-58.487762,Villa Urquiza,Departamento,3.0,2.0,80.0,68.0,250000.0,Departamento - Villa Urquiza
4,-34.598582,-58.376689,San Nicolás,Departamento,2.0,1.0,37.0,37.0,79000.0,Venta de departamento estilo frances de 2 amb...
...,...,...,...,...,...,...,...,...,...,...
16271,-34.652357,-58.385489,Barracas,Casa,5.0,4.0,223.0,223.0,295000.0,CASA EN LOTE DE 10X27 MTS 223 M2 CUB - RIO CUA...
16272,-34.653920,-58.410842,Pompeya,Casa,4.0,2.0,257.0,130.0,180000.0,VENTA-POMPEYA-CASA-2PLANTAS-DIST TECNOLOGICO
16273,-34.593420,-58.421077,Palermo,Departamento,3.0,2.0,91.0,71.0,348000.0,"Excelente oportunidad, Hermoso 3 Ambientes, nu..."
16274,-34.593404,-58.402309,Barrio Norte,Departamento,2.0,1.0,45.0,45.0,133000.0,Departamento - Barrio Norte


## Procesamiento del lenguaje natural

### Ampliación del dataset

Este dataset incluye descripciones de las propiedades del otro dataset. Veremos como podemos extraer información de estas descripciones.

In [4]:
descriptions_dataset = pd.read_csv('datasets/properati_argentina_2021_decrip.csv')

In [5]:
descriptions_dataset

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...
...,...,...
460149,MbQ4rqEzjp9Jfc3Sywb/YQ==,Corredor Responsable: Hector Garaygorta / Cris...
460150,fbuG1ohYC623nxLYPQPBvw==,Corredor Responsable: MARIA DEL ROSARIO NEME -...
460151,Ry5hMwctFXD7rLoUpzduxQ==,En cumplimiento de las leyes provinciales vige...
460152,xH37f+l+VOP8Vj0T3g+r6g==,En cumplimiento de las leyes provinciales vige...


In [6]:
# Dataset Train con ID preprocesado
train_ds_id = pd.read_csv('datasets/tp1-train_id.csv')
train_ds_id = train_ds_id.drop(['Unnamed: 0'], axis=1)
train_ds_id.head()

,id,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,AkvIZkI7XvbCQ/Z7zZ7lXw==,2021-07-29,2021-09-06 00:00:00.000000000,-34.564800,-58.444631,Belgrano,Departamento,5.0,4.0,279.0,245.0,1000000.0,Espectacular torre en la mejor zona de Belgran...
1,fPYqaGC1CO4tGa5Lfbsfxw==,2021-12-16,2022-02-09 00:00:00.000000000,-34.591359,-58.418809,Palermo,Departamento,2.0,1.0,42.0,42.0,89900.0,VENTA 2 AMBIENTES RECICLADO - PALERMO
2,iEf87GrVMz/FhIPsQTns4w==,2021-06-19,2021-09-01 15:14:13.531306906,-34.602632,-58.428399,Almagro,Departamento,2.0,1.0,46.0,42.0,119900.0,2 ambientes al frente con balcón. A estrenar.
3,po58pOgih96xIX9iBLCejA==,2021-12-16,2022-02-09 00:00:00.000000000,-34.564290,-58.439272,Palermo,Departamento,3.0,2.0,69.0,58.0,175000.0,DEPARTAMENTO 3 Ambientes Palermo-Balcón C/Parr...
4,GNPH2InKGIZ3mGgnJGK9HQ==,2021-12-16,2022-02-09 00:00:00.000000000,-34.630517,-58.377143,Barracas,Departamento,1.0,1.0,45.0,41.0,120000.0,Venta-Barracas-Mono divisible- MOCA


In [7]:
# Dataset Test con ID preprocesado
test_ds_id = pd.read_csv('datasets/tp1-test_id.csv')
test_ds_id = test_ds_id.drop(['Unnamed: 0'], axis=1)
test_ds_id.head()

,id,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,alZlUcnGhQjicsCUrLelUg==,NaN,NaN,-34.589649,-58.434926,Palermo,Casa,4.0,3.0,140.0,140.0,530000.0,Casa - Palermo
1,MizJ7379YN8pr1oUXkuoJQ==,NaN,NaN,-34.598508,-58.443084,Villa Crespo,Departamento,1.0,1.0,33.0,30.0,78000.0,Monoambiente Departamento a Estrenar - Villa C...
2,WjEJRWaB6ZSmhsePTNh9Cg==,NaN,NaN,-34.582098,-58.448859,Palermo,Departamento,1.0,1.0,35.0,30.0,87100.0,Venta - Newbery Jorge 3445 3º I Jardin
3,zTFHwNW12MiipUveEst26g==,NaN,NaN,-34.569662,-58.487762,Villa Urquiza,Departamento,3.0,2.0,80.0,68.0,250000.0,Departamento - Villa Urquiza
4,UajHnX8NrKB2DNhHfsNi5Q==,NaN,NaN,-34.598582,-58.376689,San Nicolás,Departamento,2.0,1.0,37.0,37.0,79000.0,Venta de departamento estilo frances de 2 amb...


In [8]:
delta_id_desc_props = pd.merge(left=descriptions_dataset, right=train_ds_id, left_on="id", right_on="id", how ="inner")

In [9]:
delta_id_desc_props

,id,property_description,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo
1,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...,2021-05-02,2021-06-05 00:00:00.000000000,-34.552689,-58.472921,Nuñez,PH,4.0,3.0,154.0,93.0,260000.0,PH - Nuñez
2,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,...",2021-05-02,2021-05-13 00:00:00.000000000,-34.557705,-58.471938,Coghlan,PH,5.0,4.0,145.0,110.0,279000.0,"UNICO PH LUMINOSO, TOTALM. RECICLADO- Coghlan ..."
3,+9uGr8tPEUkKnpjOef+gUw==,SE VENDE EXCELENTE PH de 2 CÓMODOS AMBIENTES M...,2021-05-02,2021-05-04 00:00:00.000000000,-34.634328,-58.489813,Floresta,PH,2.0,1.0,58.0,47.0,97500.0,"OLMOS 52: impecable PH de 2amb. más patio, apt..."
4,uH5H+RfQM87Qtrb13+T4hA==,"Venta PH 4 ambientes Paternal<br><br>Cálido, ...",2021-05-02,2021-06-05 00:00:00.000000000,-34.603158,-58.475271,Paternal,PH,4.0,2.0,127.0,89.0,145000.0,PH - Paternal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69688,5L/7wLP2/8GjJAtomb3MkQ==,Corredor Responsable: Martin Macagno - CUCICBA...,2021-09-29,2021-09-30 00:00:00.000000000,-34.642441,-58.474000,Parque Avellaneda,Departamento,3.0,2.0,68.0,64.0,109900.0,PH 3 AMBIENTES AMPLIO Y LUMINOSO SIN EXPENSAS
69689,8EZIPq3Vb+y+XH7mUgT0cw==,Corredor Responsable: Martin Macagno - CUCICBA...,2021-09-29,2021-09-30 00:00:00.000000000,-34.642441,-58.474000,Parque Avellaneda,PH,3.0,2.0,68.0,64.0,109900.0,PH 3 AMBIENTES AMPLIO Y LUMINOSO SIN EXPENSAS
69690,KfWSMnexlkmkhAXueyCVDA==,RETASADO <br>Pareja 3235 edificio doble frente...,2021-09-29,2022-03-09 00:00:00.000000000,-34.592628,-58.503425,Villa Pueyrredón,Departamento,5.0,4.0,156.0,127.0,379000.0,Departamento excelente edificio - Villa Pueyrr...
69691,z6IALPHlnIdXzLsoM2bKJQ==,*** AV. JUAN B JUSTO entre SÁNCHEZ y TRELLES *...,2021-09-29,2022-03-09 00:00:00.000000000,-34.608320,-58.464460,Villa General Mitre,Departamento,5.0,4.0,150.0,150.0,550000.0,Departamento - Villa Gral.Mitre


#### Tecnica Minqing Hu y Bing Liu

Una forma de analizar el sentimiento de un de un texto es considerando su sentimiento como la suma de los sentimientos de cada una de las palabras que lo forman.

Para el analisis de sentimiento nos guiamos del analisis realizado en esta pagina: https://www.cienciadedatos.net/documentos/py25-text-mining-python.html

Utilizamos algunas funciones de tokenizacion y limpieza de ahi con alguna sutil modificacion para nuestro caso de uso en particular.

In [10]:
def limpiar_tokenizar(texto):
    '''
    Esta función limpia y tokeniza el texto en palabras individuales.
    El orden en el que se va limpiando el texto no es arbitrario.
    El listado de signos de puntuación se ha obtenido de: print(string.punctuation)
    y re.escape(string.punctuation)
    '''
    
    # Se convierte todo el texto a minúsculas
    nuevo_texto = str(texto).lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    # Eliminación de signos de puntuación
    regex = '[\\!\\¡\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    # Tokenización por palabras individuales
    nuevo_texto = nuevo_texto.split(sep = ' ')
    # Eliminación de tokens con una longitud < 2
    nuevo_texto = [token for token in nuevo_texto if len(token) > 1]
    
    return(nuevo_texto)

In [11]:
# se aplica la función de limpieza y tokenización a cada descripcion

delta_id_desc_props['texto_tokenizado'] = delta_id_desc_props['property_description'].apply(limpiar_tokenizar)
delta_id_desc_props[['texto_tokenizado']].head()

,texto_tokenizado
0,"[corredor, responsable, maria, alejandra, geno..."
1,"[excelente, ph, en, núñez, amb, oportunidad, t..."
2,"[unico, ph, de, dormitorios, en, impecable, es..."
3,"[se, vende, excelente, ph, de, cómodos, ambien..."
4,"[venta, ph, ambientes, paternal, br, br, cálid..."


In [12]:
# unnest de la columna texto_tokenizado

descriptions_tidy = delta_id_desc_props.explode(column='texto_tokenizado')
descriptions_tidy = descriptions_tidy.drop(columns='property_description')
descriptions_tidy = descriptions_tidy.rename(columns={'texto_tokenizado':'token'})
descriptions_tidy.head()

,id,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title,token
0,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,corredor
0,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,responsable
0,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,maria
0,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,alejandra
0,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,genoveva


Notamos que las preposiciones no son relevantes para entender que atributo podria ser mejor para expandir el datast, asi que decidimos agregarlas como stopwords.

Tampoco van a variar mucho el analisis de sentimiento realizado en este trabajo.

In [13]:
## listado de stopwords

preposiciones = ["a", "ante", "bajo", "(cabe)", "con", "contra", "de", "desde", "durante", "en", "entre", "hacia", "hasta", "mediante", "para", "por", "según", "sin", "sobre", "tras", "vía"]

stop_words = []

stop_words += preposiciones

# filtrado para excluir stopwords
descriptions_tidy = descriptions_tidy[~(descriptions_tidy["token"].isin(stop_words))]

Agregamos un lexicon en español de esta pagina: https://github.com/jboscomendoza/lexicos-nrc-afinn

In [14]:
# lexicon sentimientos
lexicon = pd.read_csv('datasets/lexico_nrc.csv')
lexicon

,palabra,sentimiento,word
0,ciegamente,negativo,blindly
1,ciegamente,tristeza,blindly
2,a distancia,negativo,aloof
3,a hurtadillas,sorpresa,stealthily
4,a la deriva,anticipación,adrift
...,...,...,...
15302,zumbido,anticipación,buzz
15303,zumbido,miedo,buzz
15304,zumbido,positivo,buzz
15305,zumbido,negativo,drone


In [15]:
def mappear_valores_sentimiento(s):
    # 1 Positivo
    # 0 Neutro
    # -1 Negativo
    sentimiento_numerico = 0
    if str(s) in ['negativo', 'tristeza', 'miedo', 'enfado', 'tristeza', 'asco']:
        sentimiento_numerico = -1
    if str(s) in ['sorpresa', 'positivo', 'confianza', 'alegría']:
        sentimiento_numerico = 1
    if str(s) in ['anticipación']:
        sentimiento_numerico = 0
        
    return sentimiento_numerico
    

In [16]:
lexicon['sentimiento'] = lexicon['sentimiento'].apply(lambda x: mappear_valores_sentimiento(x))
lexicon[['sentimiento']].head()

,sentimiento
0,-1
1,-1
2,-1
3,1
4,0


In [17]:
descriptions_tidy[~descriptions_tidy.token.isin(lexicon.palabra)].head()

,id,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title,token
0,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,corredor
0,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,maria
0,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,alejandra
0,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,genoveva
0,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,bohn


In [18]:
lexicon[~lexicon.palabra.isin(descriptions_tidy.token)].head()

,palabra,sentimiento,word
0,ciegamente,-1,blindly
1,ciegamente,-1,blindly
2,a distancia,-1,aloof
3,a hurtadillas,1,stealthily
4,a la deriva,0,adrift


In [19]:
# sentimiento promedio de cada descripcion
descripcion_sentimientos = pd.merge(
                            left     = descriptions_tidy,
                            right    = lexicon,
                            left_on  = "token", 
                            right_on = "palabra",
                            how      = "inner"
                      )

descripcion_sentimientos = descripcion_sentimientos.drop(columns = ["palabra", "word"])
descripcion_sentimientos

,id,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title,token,sentimiento
0,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,responsable,1
1,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,responsable,1
2,BZCeiGkJr1WBUi6pKJQwJQ==,2021-05-02,2021-05-12 00:00:00.000000000,-34.629039,-58.423125,Boedo,PH,3.0,2.0,54.0,54.0,77000.0,Oportunidad PH 3 Ambientes en Boedo,responsable,1
3,ns8QWxBXYIdnEccdiu3qog==,2021-05-02,2021-05-12 00:00:00.000000000,-34.551132,-58.474379,Saavedra,Casa,4.0,3.0,195.0,195.0,470000.0,VENTA CASA TRIPLEX 4 AMB SAAVEDRA C/COCHERA,responsable,1
4,ns8QWxBXYIdnEccdiu3qog==,2021-05-02,2021-05-12 00:00:00.000000000,-34.551132,-58.474379,Saavedra,Casa,4.0,3.0,195.0,195.0,470000.0,VENTA CASA TRIPLEX 4 AMB SAAVEDRA C/COCHERA,responsable,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3345394,PXo/GO5I1U9YYzkzkXLGNw==,2021-12-16,2022-02-09 00:00:00.000000000,-34.608323,-58.508828,Villa del Parque,Departamento,4.0,3.0,120.0,105.0,285000.0,S /Piso 4amb dep/ coch/ Bau/ se escucha oferta...,deservicio,-1
3345395,PXo/GO5I1U9YYzkzkXLGNw==,2021-12-16,2022-02-09 00:00:00.000000000,-34.608323,-58.508828,Villa del Parque,Departamento,4.0,3.0,120.0,105.0,285000.0,S /Piso 4amb dep/ coch/ Bau/ se escucha oferta...,deservicio,-1
3345396,PXo/GO5I1U9YYzkzkXLGNw==,2021-12-16,2022-02-09 00:00:00.000000000,-34.608323,-58.508828,Villa del Parque,Departamento,4.0,3.0,120.0,105.0,285000.0,S /Piso 4amb dep/ coch/ Bau/ se escucha oferta...,deservicio,-1
3345397,PXo/GO5I1U9YYzkzkXLGNw==,2021-12-16,2022-02-09 00:00:00.000000000,-34.608323,-58.508828,Villa del Parque,Departamento,4.0,3.0,120.0,105.0,285000.0,S /Piso 4amb dep/ coch/ Bau/ se escucha oferta...,deservicio,-1


In [21]:
propiedades_sentimientos = descripcion_sentimientos[["id", "token", "sentimiento"]] \
                      .groupby(["id"])\
                      .sum().reset_index()

In [22]:
propiedades_sentimientos

,id,sentimiento
0,+++LrfX9dl/rsnbNTgVaSw==,24
1,++170jViMoXTsnGwVScNsA==,5
2,++20Jf3Ha2Epz88cJxEmJw==,10
3,++2AcJNn9KYBeuOard+DZw==,18
4,++5o0X5nwMy3BtfAvsFAaA==,-2
...,...,...
69133,zzifKawPn7V8Z63LI9+K9Q==,16
69134,zziqoG25k9M8rIgzpKwFwg==,29
69135,zzjbYucw7Rj0OtPfDWrsuA==,15
69136,zzkHuJd+M8QH9JSoKm9e0A==,19


In [23]:
propiedades_sentimientos.sentimiento.max()

216

In [57]:
top5_positivas = propiedades_sentimientos[propiedades_sentimientos['sentimiento'] > 183].head(5)

In [59]:
top5_positivas

,id,sentimiento
11085,8HZPdHHB8QMCi2U/iJkYmw==,185
20651,HApisqgDT/EcQsf/Z9pwSA==,185
27462,NX8OeQx/kxiBz8ehRtMggw==,185
28846,Ordgp/sLvmhD+RwAtErcvw==,185
36224,Vi4Kbdne9us6XT3kpmSmgw==,185
49161,hWcrJ6XkTCTGVo80jBvI1g==,185
64035,vJRa8QKk0d9i9L4Rk6Bvng==,216


In [60]:
descriptions_dataset[descriptions_dataset.id.isin(top5_positivas.id)].property_description

135535    Corredor Responsable: RMX 1 SAS - Lucas Norman...
291232    Corredor Responsable: Martin Macagno - CUCICBA...
442859    Corredor Responsable: RMX 1 SAS - Lucas Norman...
442883    Corredor Responsable: RMX 1 SAS - Lucas Norman...
442886    Corredor Responsable: RMX 1 SAS - Lucas Norman...
442903    Corredor Responsable: RMX 1 SAS - Lucas Norman...
451718    Corredor Responsable: RMX 1 SAS - Lucas Norman...
Name: property_description, dtype: object

In [26]:
propiedades_sentimientos.sentimiento.min()

-37

In [68]:
top5_negativas = propiedades_sentimientos[propiedades_sentimientos['sentimiento'] < -20].head(5)

In [69]:
descriptions_dataset[descriptions_dataset.id.isin(top5_negativas.id)].property_description

73572     <br>Casa en lote propio de 8,66 m x 11,5 Orien...
104195    Corredor Responsable: JUAN FRANCISCO CARATTINO...
208313    Emprendimiento Araoz 1937  "En La Vereda del S...
378052    Araoz 1937   "EN LA VEREDA DEL SOL"<br><br>\t\...
400729    Corredor Responsable: Patricia Maria Sodor - C...
Name: property_description, dtype: object

In [30]:
## A partir de tener el top10, evaluar si esos atributos nos sirven para expandir el dataset
## Pensar en hacer un score con la completitud de la descripcion de cada propiedad -> property_desc_score

Como es claro, la gente que publica la venta de una propiedad va a tratar de expresar la mejor publicacion y descripcion posible. Es por eso que tenemos una tasa altisima de positividad. No buscamos hacer un analisis real de a

In [31]:
def perfil_sentimientos(df):
    print(f"Positivos: {round(100 * np.mean(df.sentimiento > 0), 2)}")
    print(f"Neutros  : {round(100 * np.mean(df.sentimiento == 0), 2)}")
    print(f"Negativos: {round(100 * np.mean(df.sentimiento < 0), 2)}")
    print(" ")

perfil_sentimientos(descripcion_sentimientos)

Positivos: 64.68
Neutros  : 9.49
Negativos: 25.83
 


#### Tecnica Regex

Revisamos las siguientes paginas para entender cuales son los ammenities mas buscados en CABA, y en Argentina en general.

https://www.iprofesional.com/negocios/371702-cuales-son-los-amenities-mas-exoticos-de-edificios-en-argentina

https://www.baenegocios.com/sociedad/Ranking-de-amenities-los-servicios-que-mas-pesan-al-comprar-una-propiedad-20220119-0068.html

https://www.forbesargentina.com/negocios/amenities-servicios-mas-demandados-argentinos-comprar-una-propiedad-n11901

Dichos ammenities parecen hacer que la propiedad cotice entre un 15% y un 20% más que el precio de venta.

Sacando un promedio y haciendo un top-5 ranking, podemos notar que los mas relevantes son:

- Garage/Estacionamiento
- Pileta
- Jardin/Espacio al aire libre
- Parrilla
- SUM (Gimnasio/Spa/Sauna)

Al buscar estos datos, podriamos tratar de entender si el precio resulta mayor, contra una propiedad de similares caracteristicas pero sin estos ammenities y a partir de eso, entender que % varía del precio de venta original.

In [32]:
garage_sin_espacio = re.compile(r"[^\s]+garage|garaje|estacionamiento|parking")
garage_con_espacio = re.compile(r"[^\s]+ garage|garaje|estacionamiento|parking")

In [33]:
frecuencia_garage = {'garage': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(garage_sin_espacio, str(desc).lower())) > 0):
        frecuencia_garage['garage'] += 1
    if (len(re.findall(garage_con_espacio, str(desc).lower())) > 0):
        frecuencia_garage['garage'] += 1
print(frecuencia_garage)

{'garage': 65719}


In [34]:
pileta_sin_espacio = re.compile(r"[^\s]+pileta")
pileta_con_espacio = re.compile(r"[^\s]+ pileta")

In [35]:
frecuencia_pileta= {'pileta': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(pileta_sin_espacio, str(desc).lower())) > 0):
        frecuencia_pileta['pileta'] += 1
    if (len(re.findall(pileta_con_espacio, str(desc).lower())) > 0):
        frecuencia_pileta['pileta'] += 1
print(frecuencia_pileta)

{'pileta': 81058}


In [36]:
jardin_sin_espacio = re.compile(r"[^\s]+jardin|espacio verde")
jardin_con_espacio = re.compile(r"[^\s]+ jardin|espacio verde")

In [37]:
frecuencia_jardin = {'jardin': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(jardin_sin_espacio, str(desc).lower())) > 0):
        frecuencia_jardin['jardin'] += 1
    if (len(re.findall(jardin_con_espacio, str(desc).lower())) > 0):
        frecuencia_jardin['jardin'] += 1
print(frecuencia_jardin)

{'jardin': 33055}


In [38]:
parrilla_sin_espacio = re.compile(r"[^\s]+parrilla|bbq")
parrilla_con_espacio = re.compile(r"[^\s]+ parrilla|bbq")

In [39]:
frecuencia_parrilla = {'parrilla': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(parrilla_sin_espacio, str(desc).lower())) > 0):
        frecuencia_parrilla['parrilla'] += 1
    if (len(re.findall(parrilla_con_espacio, str(desc).lower())) > 0):
        frecuencia_parrilla['parrilla'] += 1
print(frecuencia_parrilla)

{'parrilla': 119147}


In [40]:
zoom_sin_espacio = re.compile(r"[^\s]+zoom|sum|gimansio|spa")
zoom_con_espacio = re.compile(r"[^\s]+ zoom|sum|gimansio|spa")

In [41]:
frecuencia_zoom = {'sum': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(zoom_sin_espacio, str(desc).lower())) > 0):
        frecuencia_zoom['sum'] += 1
    if (len(re.findall(zoom_con_espacio, str(desc).lower())) > 0):
        frecuencia_zoom['sum'] += 1
print(frecuencia_zoom)

{'sum': 337200}


Notamos que muchas obervaciones tenian el dato de si contaban o no con balcon asi que decidimos agregarlo al analisis para poder ver si encontramos una correlacion que nos de mas informacion con lo que respecta al balcon y a la superifice total versus la superificie cubierta y el precio de venta de cada vivienda.

In [42]:
balcon_sin_espacio = re.compile(r"[^\s]+balcon")
balcon_con_espacio = re.compile(r"[^\s]+ balcon")

In [43]:
frecuencia_balcon= {'balcon': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(balcon_sin_espacio, str(desc).lower())) > 0):
        frecuencia_balcon['balcon'] += 1
    if (len(re.findall(balcon_con_espacio, str(desc).lower())) > 0):
        frecuencia_balcon['balcon'] += 1
print(frecuencia_balcon)

{'balcon': 46510}


Otro factor interesante que podriamos analizar es a que tipo de vivienda pertenece cada ammenity. Y si donde encontramos una ammenity en particular, encontramos consecuentemente otra. Por ejemplo, una casa con jardin y parrila y/o pileta.
De esta manera podriamos tratar de determinar el costo de cada ammenity o como afecta al precio.

#### Tecnica Open Information Extraction

#### Modelos

## Redes Neuronales

### Regresión

### Regresión

### Clasificación

#### Construcción del target

#### Modelo

## Ensambles de modelos

### Ensamble 1

### Ensamble 2

## Conclusiones